# Сглаживание

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Возьмем простого робота:

In [ ]:
def three_chain(q, l):
    base = Transform.identity()
    shoulder = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1)) *
        Quaternion.from_angle_axis(q[1], Vector(0, 1, 0))
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[1]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0))
    )
    flange = elbow + Transform(
        Vector(0, 0, l[2]),
        Quaternion.identity()
    )
    return [
        base,
        shoulder,
        elbow,
        flange
    ]

In [ ]:
def wrap_from_to(value, s, e):
    r = e - s
    return value - (r * np.floor((value - s) / r))

def three_ik(target, l, i=[1, 1, 1], math_source=np):
    target = target + Vector(0, 0, -l[0])
    projection = Vector(target.x, target.y, 0)
    q0 = Vector(0, 1, 0).angle_to(projection, Vector(0, 0, 1)) - math_source.pi / 2 * i[0] + math_source.pi
    d = target.magnitude()
    q2 = -i[1] * np.arccos(
        (l[1] **  2 + l[2] ** 2 - d ** 2) /\
        (2 * l[1] * l[2])
    ) + math_source.pi
    triangle_angle = i[1] * math_source.arccos(
        (l[1] ** 2 + d ** 2 - l[2] ** 2) /\
        (2 * l[1] * d)
    )
    lift_angle = math_source.arctan2(
        target.z,
        projection.magnitude()
    )
    q1 = -i[0] * (-math_source.pi / 2 + triangle_angle + lift_angle)
    return (
        q0,
        q1,
        q2
    )

In [ ]:
three_l = [2, 3, 2.5]

In [ ]:
s = Vector(3, 0, 2)
i = Vector(3, 2, 5)
e = Vector(-1, 4, 3)
blend = 0.25

## Линейное-линейное

In [ ]:
def lin(l, start, end, t, total):
    target = Vector.lerp(
        start,
        end,
        t / total
    )
    return three_ik(target, l)
    

def lin_lin(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return lin(l, start, inter, progress, 0.5)
    return lin(l, inter, end, progress - 0.5, 0.5)

def lin_lin_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = Vector.lerp(start, inter, 1.0 - 2 * blend)
        b = inter
        c = Vector.lerp(inter, end, 2 * blend)
        trg = (1 - progress) ** 2 * a + 2 * progress * (1 - progress) * b + progress ** 2 * c
        return three_ik(trg, l)
    return lin_lin(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    three_chain([0, 0, 0], three_l)
)
fig, ax = graphics.figure(10)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, Transform(s, Quaternion.identity()), 1)
graphics.axis(ax, Transform(i, Quaternion.identity()), 1)
graphics.axis(ax, Transform(e, Quaternion.identity()), 1)

total = 100

def animate(frame):
    q = lin_lin_smooth(three_l, s, i, e, frame, total, blend)
    chain = three_chain(q, three_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_lin_lin_smooth = np.vectorize(lin_lin_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_lin_lin_smooth(three_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(three_chain(q, three_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.legend()

aw.legend()
fig.show()

## Переброска-переброска

In [ ]:
def ptp(l, start, end, t, total):
    s = three_ik(start, l)
    e = three_ik(end, l)
    q = (
        s[0] + (e[0] - s[0]) * t / total,
        s[1] + (e[1] - s[1]) * t / total,
        s[2] + (e[2] - s[2]) * t / total
    )
    return q

def ptp_ptp(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return ptp(l, start, inter, progress, 0.5)
    return ptp(l, inter, end, progress - 0.5, 0.5)

def ptp_ptp_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = ptp(l, start, inter, 1.0 - 2 * blend, 1)
        b = three_ik(inter, l)
        c = ptp(l, inter, end, 2 * blend, 1)
        return (
            a[0] * (1 - progress) ** 2 + b[0] * 2 * progress * (1 - progress) + c[0] * progress ** 2,
            a[1] * (1 - progress) ** 2 + b[1] * 2 * progress * (1 - progress) + c[1] * progress ** 2,
            a[2] * (1 - progress) ** 2 + b[2] * 2 * progress * (1 - progress) + c[2] * progress ** 2,
        )
    return ptp_ptp(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    three_chain([0, 0, 0], three_l)
)
fig, ax = graphics.figure(10)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, Transform(s, Quaternion.identity()), 1)
graphics.axis(ax, Transform(i, Quaternion.identity()), 1)
graphics.axis(ax, Transform(e, Quaternion.identity()), 1)

total = 100

def animate(frame):
    q = ptp_ptp_smooth(three_l, s, i, e, frame, total, blend)
    chain = three_chain(q, three_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_ptp_ptp_smooth = np.vectorize(ptp_ptp_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_ptp_ptp_smooth(three_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(three_chain(q, three_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.legend()

aw.legend()
fig.show()

## Линейное-переброска

Наивная реализация

In [ ]:
def lin_ptp(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return lin(l, start, inter, progress, 0.5)
    return ptp(l, inter, end, progress - 0.5, 0.5)

def lin_ptp_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        
        l_blend_i = inter
        q_blend_i = three_ik(inter, l)
        
        l_blend_s = Vector.lerp(start, inter, 1 - 2 * blend)
        q_blend_s = three_ik(l_blend_s, l)
        
        q_blend_e = ptp(l, inter, end, 2 * blend, 1)
        l_blend_e = three_chain(q_blend_e, l)[-1].translation
        
        l_blend = three_ik(
            Vector.lerp(
                Vector.lerp(l_blend_s, l_blend_i, progress),
                Vector.lerp(l_blend_i, l_blend_e, progress),
                progress
            ),
            l)
        q_blend = (
            q_blend_s[0] * (1 - progress) ** 2 + q_blend_i[0] * 2 * (1 - progress) * progress + q_blend_e[0] * progress ** 2,
            q_blend_s[1] * (1 - progress) ** 2 + q_blend_i[1] * 2 * (1 - progress) * progress + q_blend_e[1] * progress ** 2,
            q_blend_s[2] * (1 - progress) ** 2 + q_blend_i[2] * 2 * (1 - progress) * progress + q_blend_e[2] * progress ** 2,
        )
        return (
            l_blend[0] + (q_blend[0] - l_blend[0]) * progress,
            l_blend[1] + (q_blend[1] - l_blend[1]) * progress,
            l_blend[2] + (q_blend[2] - l_blend[2]) * progress,
        )
    return lin_ptp(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    three_chain([0, 0, 0], three_l)
)
fig, ax = graphics.figure(10)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, Transform(s, Quaternion.identity()), 1)
graphics.axis(ax, Transform(i, Quaternion.identity()), 1)
graphics.axis(ax, Transform(e, Quaternion.identity()), 1)

total = 100

def animate(frame):
    q = lin_ptp_smooth(three_l, s, i, e, frame, total, blend)
    chain = three_chain(q, three_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(50)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_lin_ptp_smooth = np.vectorize(lin_ptp_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_lin_ptp_smooth(three_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(three_chain(q, three_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.legend()

aw.legend()
fig.show()